In [ ]:
# Se connecter à une DB existente 

import sqlite3 
import pandas as pd
import matplotlib.pyplot as plt

def initDB (db_path) :
    
    try:
        sqliteConnection = sqlite3.connect('billboard/billboard-200.db')
        print("Database created and Successfully Connected to SQLite")

    except sqlite3.Error as error:
        print("Error while connecting to sqlite", error)
    
    return sqliteConnection
    
def sqlQuery (sqliteConnection, query) :
    
    if sqliteConnection :
        cursor = sqliteConnection.cursor()
        sqlite_select_Query = query
        cursor.execute(sqlite_select_Query)
        record = cursor.fetchall()
        cursor.close()
        return record
    else : 
        print("DB connection KO")
        return 'KO'
        
sqliteConnection = initDB('billboard/billboard-200.db')

values = sqlQuery (sqliteConnection, """SELECT artist, COUNT(artist) AS whala 
                      FROM albums
                      WHERE (rank=1) AND (date > 2000)
                      GROUP BY artist, album
                      ORDER BY whala DESC
                      LIMIT 10 """)


In [ ]:
# Fermer la connection

sqliteConnection.close()
print("The SQLite connection is closed")


In [ ]:
# Récupérer requête dans un DataFrame Pandas

conn = initDB('billboard/billboard-200.db')
query = """SELECT * FROM acoustic_features"""

mydb = pd.read_sql(query, con=conn)
mydb.set_index(pd.to_datetime(mydb.date), inplace=True)
means = mydb.groupby(mydb.index.year).mean()

with plt.style.context('fivethirtyeight'):
    fig, axes = plt.subplots(4, 4, figsize=(30, 20), sharex=True)
    for i, col in enumerate(means):
        axes[i//4, i%4].plot(means[col])
        axes[i//4, i%4].set_title(col)


In [ ]:
# Quelle est l’année dont le niveau sonore «loudness » a été le plus haut ? 
mydb['loudness'].idxmax().year

In [ ]:
# Quelle est la clé musicale la plus populaire - en prenant en compte le mode (e.g. majeur,  mineur) ? 

conn = initDB('billboard/billboard-200.db')
query = """SELECT  f.key, f.mode, COUNT(*) as nb
           FROM acoustic_features f 
           GROUP BY f.key, f.mode
           ORDER BY COUNT(*) DESC"""

result = pd.read_sql(query, con=conn)
result


# Itération 3 

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import re

def read_gtfs_file (file_name):
    df = pd.read_csv(file_name, index_col=None, header=None, na_values='ND')
    return df

def format_line_for_SQL_insert (df) :

    valuesToInsert = df.values.tolist()
    valuesToInsert.pop(0)
    listVal = []
    for obs in range(0,len(valuesToInsert)) :
        val = re.sub('\[','',str(valuesToInsert[obs])) 
        val = re.sub('\]','',val)
        listVal.append(val)
        
        
    return listVal

def insert_data_SQL (listData, bdd, table) :
    
    con = sqlite3.connect(bdd)
    cur = con.cursor()

    for data in listData :
        print(data)
        # Insert a row of data
        cur.execute(f"INSERT INTO {table} VALUES ({data})")
        con.commit()
    
    con.close()
    
def gen_insert_file(filename, tablename, df) :
    
    sqlFile = open(filename, "w")
    sqlFile.write("BEGIN")
    for data in listData :
        sqlFile.write(f"INSERT INTO {tablename} VALUES ({data})")
    sqlFile.write("END")
    sqlFile.close()
    
    

bdd = './Iteration3/test'
agencyData = './Iteration3/data/agency.txt'
stopsData = './Iteration3/data/stops.txt'

dfAgency = read_gtfs_file(agencyData)
dfStops = read_gtfs_file(stopsData)

dfStops = dfStops.replace(np.nan, 'nan')

agencyValues = format_line_for_SQL_insert(dfAgency)
stopsValues = format_line_for_SQL_insert(dfStops)

insert_data_SQL(stopsValues, bdd, 'stops')


'0004', '0004', 'Meylan, Revirée - Sayettes', '45.2122', '5.76728', 'nan', 'nan', 'REVIREES', '0'


OperationalError: database is locked

In [ ]:
stopsValues